In [1]:
import json
import time
import sys
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# 셀레눔 드라이버와 스크롤 횟수를 받아서 스크롤을 내리는 함수 
def scroll_page(driver, scroll_count):  # 스크롤 횟수만큼 스크롤을 내림
    for _ in range(scroll_count):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)

In [16]:
def extract_reviews(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select(".review-item-container")
    data = []

    for item in items:
        if item:
            # 별점 추출
            review_point = len(item.find_all('path', attrs={'fill': '#FDBD00'}))
            
            # 리뷰 텍스트 추출
            review_text = item.select_one(".content-text.css-c92dc4").text.strip()

            # 리뷰 날짜 추출 (선택자 변경: css-1irbwe1)
            date_element = item.select_one("p.css-1irbwe1")
            review_date = date_element.text.strip() if date_element else "날짜 정보 없음"

            # JSON 형식에 맞게 딕셔너리 추가
            data.append({
                "review": review_text,
                "stars": review_point,
                "date": review_date
            })
    return data

In [17]:
# 리뷰를 엑셀 및 JSON으로 저장하는 함수
def save_reviews(name, review_list): 
    # 엑셀로 저장
    df = pd.DataFrame(review_list)
    df_unique = df.drop_duplicates()
    df_unique.to_excel(f"yanolja-review\\review_list_{name}.xlsx", index=False)
    
    # JSON으로 저장
    with open(f"yanolja-review\\review_list_{name}.json", 'w', encoding='utf-8') as json_file:
        json.dump(review_list, json_file, ensure_ascii=False, indent=4)
    return

In [18]:
def crawl_yanolja_reviews(name, url):
    # 드라이버 
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(3)

    # 스크롤 횟수
    scroll_count = 20
    # 스크롤 내리기 함수 실행
    scroll_page(driver, scroll_count)

    # 리뷰 추출
    html = driver.page_source
    # 리뷰 추출 함수 실행
    review_list = extract_reviews(html)

    # 리뷰 저장 함수 실행 
    save_reviews(name, review_list)
    driver.quit()

In [19]:
if __name__ == '__main__':
    name="리치몬드 호텔 텐진 니시-도리"
    url="https://www.yanolja.com/reviews/global/2513843"
    crawl_yanolja_reviews(name=name, url=url)